## Import library

In [1]:
import sys

MAIN_PATH = "/home/trduong/Data/interpretable_machine_learning/Source Code/my_work"


sys.path.insert(
    1,
    MAIN_PATH + '/lib'
)

sys.path.insert(
    1,
    MAIN_PATH + '/config'
)

import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import incremental_ps_score_estimator as ipse
import experiment_config as cf
import math
import timeit
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import utils 
import experiment_config as cf

from tempfile import TemporaryFile

# save numpy array as npy file
from numpy import asarray
from numpy import save

from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import logging
import pygad


/home/trduong/anaconda3/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/trduong/anaconda3/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
tf.__version__

'2.2.0'

In [3]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"]

## Optimization with Adam and original function

In [10]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence)

def optimization(data):
    threhold = tf.constant([0.001])
    delta = tf.Variable(100., trainable = True)
    true_effect = np.mean(data['mu1'] - data['mu0'])
    
    for i in range(150000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            ## early stopping
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

### Optimize in train data

In [ ]:
delta_seq = []
losses = []
losses_test = []
for index_ in (range(1, 101)):
    data = utils.load_data(cf.IHDP_TRAIN, index_)
    data_test = utils.load_data(cf.IHDP_TEST, index_)
    data_whole = pd.concat([data, data_test], sort=False)
    #data, data_test = train_test_split(data_whole, test_size=0.1)
    data = data.reset_index()
    data_test = data_test.reset_index()
    #oversample = SMOTE()
    #X_train_oversampled, y_train_oversampled = oversample.fit_sample(data_whole[cov].values, data_whole[treatment].values)

    ## Fit treatment
    print("Fit treament")

    model_t = LogisticRegression()
    model_t.fit(data_whole[cov], data_whole[treatment])

    ## Fit outcome
    print("Fit outcome")

    model_y = GradientBoostingRegressor(random_state=0, n_estimators=2000)
    model_y.fit(data_whole[features], data_whole[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:, 1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    print("Compute counterfactual outcome")

    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] +
                          (1 - data[treatment]) / (1 - data['p1']))

    data['w0'] = data['ips_weight'] * data[treatment]
    data['w1'] = data['ips_weight'] * (1 - data[treatment])

    print("Optimization")

    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    losses.append(loss.numpy())

    influence = ipse.influence_function(data_test, treatment, cov, outcome,
                                        features, delta_r, model_y, model_t)
    means_incre_test, stds_incre = np.mean(influence, axis=0), sem(influence,
                                                                   axis=0)
    
    print("Calculate ATE in the testing set")
    true_effect = data['mu1'] - data['mu0']
    means_train, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)

    true_effect = data_test['mu1'] - data_test['mu0']
    means_test, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)

    mae_test = utils.abs_ate(means_test, means_incre_test)
    losses_test.append(mae_test)
    print("\n")
    print("Data index {}. Delta {:.2f}".format(index_, delta_r))
    print("MAE on traning {:.2f}".format(loss.numpy()))
    print("MAE on testing {:.2f}".format(mae_test))
    print("*" * 100)

Fit treament
Fit outcome
Compute counterfactual outcome
Optimization
Calculate ATE in the testing set


Data index 1. Delta 248.12
MAE on traning 0.10
MAE on testing 0.06
****************************************************************************************************
Fit treament
Fit outcome
Compute counterfactual outcome
Optimization
Calculate ATE in the testing set


Data index 2. Delta 247.95
MAE on traning 0.16
MAE on testing 0.07
****************************************************************************************************
Fit treament
Fit outcome
Compute counterfactual outcome
Optimization
Calculate ATE in the testing set


Data index 3. Delta 248.33
MAE on traning 0.14
MAE on testing 0.33
****************************************************************************************************
Fit treament
Fit outcome
Compute counterfactual outcome
Optimization
Calculate ATE in the testing set


Data index 4. Delta 247.95
MAE on traning 0.14
MAE on testing 0.05
*************

Fit treament
Fit outcome
Compute counterfactual outcome
Optimization
The performance reach MAE: 0.001. Cancelling the training at step 92883
Calculate ATE in the testing set


Data index 28. Delta 7.16
MAE on traning 0.00
MAE on testing 3.14
****************************************************************************************************
Fit treament
Fit outcome
Compute counterfactual outcome
Optimization
The performance reach MAE: 0.001. Cancelling the training at step 48204
Calculate ATE in the testing set


Data index 29. Delta 51.82
MAE on traning 0.00
MAE on testing 0.27
****************************************************************************************************
Fit treament
Fit outcome
Compute counterfactual outcome
Optimization
Calculate ATE in the testing set


Data index 30. Delta 248.99
MAE on traning 0.05
MAE on testing 0.01
****************************************************************************************************
Fit treament
Fit outcome
Compute counter

In [ ]:
print("MAE Training {:.4f}".format(np.mean(losses)))
print("MAE Testing {:.4f}".format(np.mean(losses_test)))

In [ ]:
# MAE Training 0.1605
# MAE Testing 0.5021

### optimize train-test together
# MAE Training 0.0933
# MAE Testing 0.6035


In [ ]:
data = asarray(delta_seq)
save('delta_.npy', delta_seq)

In [ ]:
PATH_TRAIN = "/home/dtd/Documents/data/idhp/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/data/idhp/ihdp_npci_1-100.test.npz"

delta_seq = []
losses = []
losses_test = []
for index_ in tqdm(range(1, 101)): 
    data = utils.load_data(PATH_TRAIN, index_)
    data_test = utils.load_data(PATH_TEST, index_)
    data_whole = pd.concat([data, data_test], sort=False)
    
    #oversample = SMOTE()
    #X_train_oversampled, y_train_oversampled = oversample.fit_sample(data_whole[cov].values, data_whole[treatment].values)
    
    ## Fit treatment
    model_t = LogisticRegression()
    #model_t.fit(X_train_oversampled, y_train_oversampled)

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 500)
    model_y.fit(data_whole[features], data_whole[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    losses.append(loss.numpy())
    
    
    influence = ipse.influence_function(data_test, treatment, cov, outcome, features, delta_r, model_y, model_t)
    means_incre_test, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

    true_effect = data['mu1'] - data['mu0']
    means_train, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    
    true_effect = data_test['mu1'] - data_test['mu0']
    means_test, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)

    mae_test = utils.abs_ate(means_test, means_incre_test)
    losses_test.append(mae_test)
    print("\n")
    print("Data index {}. Delta {:.2f}".format(index_, delta_r))
    print("MAE on traning {:.2f}".format(loss.numpy()))
    print("MAE on testing {:.2f}".format(mae_test))
    print("*"*100)

## Optimization with list of delta

In [4]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence)

def optimization(data, variation):
    threhold = tf.constant([0.001])
    '''
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=1, 
                          maxval=100, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    '''
    delta = tf.Variable(tf.random.normal(
        [data.shape[0],], 
        mean=0.0, 
        stddev=variation, 
        dtype=tf.dtypes.float32, 
        seed=1, 
        name='delta'
    ), trainable = True)
    
    true_effect = np.mean(data['mu1'] - data['mu0'])
    
    for i in range(30000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            ## early stopping
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

### Optimize in train data

In [ ]:
delta_seq = []
losses = []
losses_test = []

seq_result = pd.DataFrame()
seq_result['train'] = 0
seq_result['test'] = 0

for count in range(1,100):
    #variation = np.random.randint(low=1, high=100, size=1)
    print("Variaton ", count)
    for index_ in range(1, 101): 
        data = utils.load_data(cf.IHDP_TRAIN, index_)
        data_test = utils.load_data(cf.IHDP_TEST, index_)
        data_whole = pd.concat([data, data_test], sort=False)
        ## Fit treatment
        print("Fit treament")
        model_t = LogisticRegression()
        model_t.fit(data_whole[cov], data_whole[treatment])

        ## Fit outcome
        print("Fit outcome")
        model_y = GradientBoostingRegressor(random_state=0, n_estimators = 1000)
        model_y.fit(data_whole[features], data_whole[outcome])

        data['p1'] = model_t.predict_proba(data[cov])[:,1]
        data_test['p1'] = model_t.predict_proba(data_test[cov])[:,1]

        data['p0'] = 1 - data['p1']

        ## Compute counterfactual outcome with no treatment
        print("Compute counterfactual outcome")
        data_pos = data.copy()
        data_pos[treatment] = 1
        data['cf1'] = model_y.predict(data_pos[features])

        ## Compute counterfactual outcome with treatment
        data_neg = data.copy()
        data_neg[treatment] = 0
        data['cf0'] = model_y.predict(data_neg[features])

        data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                              (1 - data['p1']))

        data['w0'] = data['ips_weight']*data[treatment]
        data['w1'] = data['ips_weight']*(1 - data[treatment])
        
        print("Optimization")
        delta, loss = optimization(data, count)
        delta_r = delta.numpy()
        delta_seq.append(delta_r)
        losses.append(loss)
        
        treated_neighbors = (
                NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
                .fit(data['p1'].values.reshape(-1, 1))
        )
        distances, indices = treated_neighbors.kneighbors(data_test['p1'].values.reshape(-1, 1))

        delta_test = delta_r[indices.reshape(-1)]
        
        print("Calculate ATE in the testing set")
    
        influence = ipse.influence_function(data_test, 
                                            treatment, 
                                            cov, 
                                            outcome, 
                                            features,
                                            delta_test, 
                                            model_y, 
                                            model_t)

        means_incre_test, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

        ## True effect   
        
        true_effect = data_test['mu1'] - data_test['mu0']
        means_test, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
        
        print("Calculate mean absolute error")
        mae_test = utils.abs_ate(means_test, means_incre_test)
        losses_test.append(mae_test)

        print("Data index {}. Loss training {:.2f} and testing {:.2f}".format(index_, loss.numpy(), mae_test))

        seq_result.loc[index_ - 1, 'train'] = loss.numpy()
        seq_result.loc[index_ - 1, 'test'] = mae_test
        print("*"*100)
    print("Export to csv")
    print("-"*100)
    
    save(cf.RESULT_PATH + "/" + "list_delta_" + str(index_) + ".npy", delta_seq)
    seq_result.to_csv(cf.RESULT_PATH + "/" + str(count) + "_ps_con.csv")


Variaton  1
Fit treament
Fit outcome
Compute counterfactual outcome
Optimization
Calculate ATE in the testing set
Calculate mean absolute error
Data index 1. Loss training 0.04 and testing 0.34
****************************************************************************************************
Fit treament
Fit outcome
Compute counterfactual outcome
Optimization
Calculate ATE in the testing set
Calculate mean absolute error
Data index 2. Loss training 0.01 and testing 0.28
****************************************************************************************************
Fit treament
Fit outcome
Compute counterfactual outcome
Optimization
Calculate ATE in the testing set
Calculate mean absolute error
Data index 3. Loss training 0.02 and testing 0.71
****************************************************************************************************
Fit treament
Fit outcome
Compute counterfactual outcome
Optimization
Calculate ATE in the testing set
Calculate mean absolute error
Data in






## Optimization for individual treatment effect

In [ ]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return influence

def optimization(data):
    threhold = tf.constant([0.01])
    '''
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=1, 
                          maxval=100, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    '''
    delta = tf.Variable(tf.random.normal(
        [data.shape[0],], 
        mean=0, 
        stddev=10, 
        dtype=tf.dtypes.float32, 
        seed=1, 
        name='delta'
    ), trainable = True)
    
    true_effect = data['mu1'] - data['mu0']
    
    for i in range(100000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.keras.losses.MAE(true_effect, influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            if i % 1000 == 0:
                print("Epoch {}. Loss {:.5f}".format(i, loss.numpy()))
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

In [ ]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
for index_ in tqdm(range(13, 14)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    print("Data index {}. Loss {:.2f}".format(index_, loss.numpy()))

In [ ]:
save('individual_list_delta_.npy', delta_seq)

## Optimization for individual treatment effects using genetic algorithm

In [ ]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return influence


def fitness_function(delta, solution_idx):
    influence = incre_ps(delta, data)
    mae = -np.sqrt(np.mean((true_effect - influence)**2))
    return np.mean(mae)



In [ ]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
data = utils.load_data(PATH_TRAIN, 1)

## Fit treatment
model_t = LogisticRegression()
model_t.fit(data[cov], data[treatment])

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(data[features], data[outcome])

data['p1'] = model_t.predict_proba(data[cov])[:,1]
data['p0'] = 1 - data['p1']

## Compute counterfactual outcome with no treatment
data_pos = data.copy()
data_pos[treatment] = 1
data['cf1'] = model_y.predict(data_pos[features])

## Compute counterfactual outcome with treatment
data_neg = data.copy()
data_neg[treatment] = 0
data['cf0'] = model_y.predict(data_neg[features])

data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                      (1 - data['p1']))

data['w0'] = data['ips_weight']*data[treatment]
data['w1'] = data['ips_weight']*(1 - data[treatment])


In [ ]:
true_effect = data['mu1'] - data['mu0']

In [ ]:

"""
Given the following function:
    y = f(w1:w6) = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + 6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7) and y=44
What are the best values for the 6 weights (w1 to w6)? We are going to use the genetic algorithm to optimize this function.
"""

#function_inputs = [4,-2,3.5,5,-11,-4.7] # Function inputs.
#desired_output = 44 # Function output.

function_inputs = np.random.rand(data.shape[0]) # Function inputs.

num_generations = 5000 # Number of generations.
num_parents_mating = 100 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 2500 # Number of solutions in the population.
num_genes = len(function_inputs)

init_range_low = -100
init_range_high = 100

parent_selection_type = "sss" # Type of parent selection.
keep_parents = -1 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "single_point" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 10 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    print("\n")
    last_fitness = ga_instance.best_solution()[1]

# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()


## Optimization ATE using genetic algorithm

In [ ]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return influence


def fitness_function(delta, solution_idx):
    influence = incre_ps(delta, data)
    mae = -np.sqrt(np.mean((true_effect - influence)**2))
    return np.mean(mae)



In [ ]:
index_ = 1

data = utils.load_data(cf.IHDP_TRAIN, index_)
data_test = utils.load_data(cf.IHDP_TEST, index_)
data_whole = pd.concat([data, data_test], sort=False)
## Fit treatment
model_t = LogisticRegression()
model_t.fit(data_whole[cov], data_whole[treatment])

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(data_whole[features], data_whole[outcome])

data['p1'] = model_t.predict_proba(data[cov])[:,1]
data['p0'] = 1 - data['p1']

## Compute counterfactual outcome with no treatment
data_pos = data.copy()
data_pos[treatment] = 1
data['cf1'] = model_y.predict(data_pos[features])

## Compute counterfactual outcome with treatment
data_neg = data.copy()
data_neg[treatment] = 0
data['cf0'] = model_y.predict(data_neg[features])

data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                      (1 - data['p1']))

data['w0'] = data['ips_weight']*data[treatment]
data['w1'] = data['ips_weight']*(1 - data[treatment])

true_effect = data['mu1'] - data['mu0']

In [ ]:
function_inputs = np.random.rand(data.shape[0]) # Function inputs.

num_generations = 2000 # Number of generations.
num_parents_mating = 100 # Number of solutions to be selected as parents in the mating pool.

sol_per_pop = 2500 # Number of solutions in the population.
num_genes = len(function_inputs)

init_range_low = -100
init_range_high = 100

parent_selection_type = "sss" # Type of parent selection.
keep_parents = -1 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "single_point" # Type of the crossover operator.

mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 10 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.
last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    print("\n")
    last_fitness = ga_instance.best_solution()[1]
    if last_fitness <=  0.001:
        break 

# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()
